In [1]:
%load_ext autoreload
%autoreload 2

### Load a presaved DFallpa

In [2]:
import pandas as pd
#path = f"/home/lucas/Dropbox/SCIENCE/FREIWALD_LAB/DATA/DFallpa_Diego_230929_RULE_BASE_stroke.pkl"
animal = "Diego"
date = "230630"
path= f"/home/kgg/Desktop/eyetracking_analyses/{animal}-{date}/dfallpa.pkl"
DFallpa = pd.read_pickle(path)

In [3]:
DFallpa

,which_level,event,bregion,twind,pa,pa_x_shape
0,flex,fixon_preparation,M1_m,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(23, 2381, 80)"
1,flex,fixon_preparation,M1_l,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(18, 2381, 80)"
2,flex,fixon_preparation,PMv_l,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(43, 2381, 80)"
3,flex,fixon_preparation,PMv_m,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(36, 2381, 80)"
4,flex,fixon_preparation,PMd_p,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(24, 2381, 80)"
5,flex,fixon_preparation,PMd_a,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(19, 2381, 80)"
6,flex,fixon_preparation,dlPFC_a,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(27, 2381, 80)"
7,flex,fixon_preparation,vlPFC_p,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(32, 2381, 80)"
8,flex,fixon_preparation,vlPFC_a,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(18, 2381, 80)"
9,flex,fixon_preparation,FP_p,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(17, 2381, 80)"


In [ ]:
# examining amount of data per condition
from pythonlib.tools.pandastools import extract_with_levels_of_conjunction_vars_helper

extract_with_levels_of_conjunction_vars_helper(df_temp, ["seqc_0_shape"],
    #["task_kind"],
    #["seqc_0_loc_on_clust", "task_kind"],
    #["loc-fixation", "seqc_0_loc_on_clust", "task_kind"],
    #["shape-fixation", "loc-fixation", "seqc_0_loc_on_clust", "task_kind"],
    ["is-first-macrosaccade", "shape-fixation", "loc-fixation", "seqc_0_loc", "task_kind"],
    n_min_per_lev=5,
    plot_counts_heatmap_savepath="/tmp/test")

# original vars, but nmin=3
# new vars, and nmin=5

# NEW MODULATION SCORE EUCLIDIAN

In [27]:
# Plot only a subset of regions
bregions_plot = None #["PMd_a", "PMd_p"]

# Or plot all regions
# bregions_plot = None # TO plot all


### Get variables

In [28]:
# Method 2 - hand enter (for testing)

# Variable of effect
LIST_VAR = [
    "is-fixated-on-seqc0shape",
    "is-fixated-on-seqc0shape",
]

# Variables to condition on
LIST_VARS_OTHERS = [
    ["shape-macrosaccade-index", "seqc_0_shape", "seqc_0_loc", "task_kind"],
    ["shape-macrosaccade-index", "seqc_0_shape", "seqc_0_loc", "task_kind"],
]

# For purposes of testing generalization acros contexts. Leave as None to skip
# LIST_CONTEXT = [
#     {"same":["epoch"], "diff":["shape"]},
# ]
LIST_CONTEXT = None #[]

# N min levels needed to keep contexts
LIST_PRUNE_MIN_N_LEVS = [
    2 for _ in range(len(LIST_VAR))
]

# Optional: filter data (throw out trials that fail filter)
# LIST_FILTDICT = [{"task_kind": ["prims_on_grid"]}]
LIST_FILTDICT = [
    {"task_kind": ["prims_on_grid"], "shape-macrosaccade-index": [1]},
    {"task_kind": ["prims_on_grid"], "shape-macrosaccade-index": list(range(2,11))},
]
# LIST_FILTDICT = [
#     {"stroke_index": list(range(1, 10, 1))},
# ]


In [29]:
SAVEDIR_ANALYSIS = f"/home/kgg/Desktop/eyetracking_analyses/SUBSPACES/{animal}-{date}/euclidian_dist_plots_macrosacc2"
import matplotlib.pyplot as plt
import os
import pandas as pd

In [30]:
from neuralmonkey.analyses.state_space_good import euclidian_distance_compute_scalar

# PARAMS: data represnetation
SHUFFLE = False
umap_n_neighbors = 40
#NPCS_KEEP = 10

# PARAMS (Remove plots to speed up)
PLOT = False # plot raw
PLOT_MASKS = False # Plot context masks (can be slow!)
nmin_trials_per_lev = 5 # to prune data
PLOT_STATE_SPACE = True # Scatter plot of neural data -- useful!
twind_list = [(-0.2, 0), (0, 0.2)] # relative to event
tbin_dur = 0.1 # windowing
tbin_slice = tbin_dur

# changed 240506
NPCS_KEEP=6
dim_red_method="pca"

# RUN
list_dfres = []
for i, row in DFallpa.iterrows():
    for twind in twind_list:
        br = row["bregion"]
        tw = row["twind"]
        ev = row["event"]
        PA = row["pa"]

        if bregions_plot is not None:
            if br not in bregions_plot:
                continue

        savedir = f"{SAVEDIR_ANALYSIS}/{br}/{twind}"
        os.makedirs(savedir, exist_ok=True)

        dfres = euclidian_distance_compute_scalar(PA, LIST_VAR, LIST_VARS_OTHERS, PLOT, PLOT_MASKS,
                                   twind, tbin_dur, tbin_slice, savedir, PLOT_STATE_SPACE=PLOT_STATE_SPACE,
                                           nmin_trials_per_lev=nmin_trials_per_lev, LIST_CONTEXT=LIST_CONTEXT,
                                           LIST_FILTDICT=LIST_FILTDICT, NPCS_KEEP=NPCS_KEEP, dim_red_method=dim_red_method,
                                           umap_n_neighbors = umap_n_neighbors, LIST_PRUNE_MIN_N_LEVS=LIST_PRUNE_MIN_N_LEVS,
                                          extra_dimred_method_n_components=3)
        plt.close("all")
        if len(dfres)>0:
            dfres["bregion"] = br
            dfres["twind"] = [tw for _ in range(len(dfres))]
            dfres["event"] = ev
            list_dfres.append(dfres)
        
DFRES = pd.concat(list_dfres).reset_index(drop=True)

Running PCA
cumvar 0.9 0.01
RUNNING:  0 is-fixated-on-seqc0shape  --  ['shape-macrosaccade-index', 'seqc_0_shape', 'seqc_0_loc', 'task_kind']
len pa bnefore filt this values (var, levs):  task_kind ['prims_on_grid']
pa size, before slicing with variable=task_kind (6, 2381, 1)
... pa size, after slicing with variable=task_kind (6, 2000, 1)
len pa bnefore filt this values (var, levs):  shape-macrosaccade-index [1]
pa size, before slicing with variable=shape-macrosaccade-index (6, 2000, 1)
... pa size, after slicing with variable=shape-macrosaccade-index (6, 462, 1)
FINAL DIMENSION OF DATA (after dimredu, before eucl): (6, 412, 1)
... dataextract_as_distance_matrix_clusters_flex
... computing distance matrices, using distnace: euclidian_unbiased
... rsa_distmat_score_same_diff_by_context
... computing distance matrices, using distnace: euclidian
DIST_NULL_50 2.3630588861394024
fig: /home/kgg/Desktop/eyetracking_analyses/SUBSPACES/Diego-230630/euclidian_dist_plots_macrosacc2/M1_m/(-0.2, 0)

Running PCA
cumvar 0.9 0.01
RUNNING:  0 is-fixated-on-seqc0shape  --  ['shape-macrosaccade-index', 'seqc_0_shape', 'seqc_0_loc', 'task_kind']
len pa bnefore filt this values (var, levs):  task_kind ['prims_on_grid']
pa size, before slicing with variable=task_kind (6, 2381, 1)
... pa size, after slicing with variable=task_kind (6, 2000, 1)
len pa bnefore filt this values (var, levs):  shape-macrosaccade-index [1]
pa size, before slicing with variable=shape-macrosaccade-index (6, 2000, 1)
... pa size, after slicing with variable=shape-macrosaccade-index (6, 462, 1)
FINAL DIMENSION OF DATA (after dimredu, before eucl): (6, 412, 1)
... dataextract_as_distance_matrix_clusters_flex
... computing distance matrices, using distnace: euclidian_unbiased
... rsa_distmat_score_same_diff_by_context
... computing distance matrices, using distnace: euclidian
DIST_NULL_50 2.3264372886429037
fig: /home/kgg/Desktop/eyetracking_analyses/SUBSPACES/Diego-230630/euclidian_dist_plots_macrosacc2/M1_l/(-0.2, 0)

Running PCA
cumvar 0.9 0.01
RUNNING:  0 is-fixated-on-seqc0shape  --  ['shape-macrosaccade-index', 'seqc_0_shape', 'seqc_0_loc', 'task_kind']
len pa bnefore filt this values (var, levs):  task_kind ['prims_on_grid']
pa size, before slicing with variable=task_kind (6, 2381, 1)
... pa size, after slicing with variable=task_kind (6, 2000, 1)
len pa bnefore filt this values (var, levs):  shape-macrosaccade-index [1]
pa size, before slicing with variable=shape-macrosaccade-index (6, 2000, 1)
... pa size, after slicing with variable=shape-macrosaccade-index (6, 462, 1)
FINAL DIMENSION OF DATA (after dimredu, before eucl): (6, 412, 1)
... dataextract_as_distance_matrix_clusters_flex
... computing distance matrices, using distnace: euclidian_unbiased
... rsa_distmat_score_same_diff_by_context
... computing distance matrices, using distnace: euclidian
DIST_NULL_50 3.4647542834989
fig: /home/kgg/Desktop/eyetracking_analyses/SUBSPACES/Diego-230630/euclidian_dist_plots_macrosacc2/PMv_l/(-0.2, 0)/c

Running PCA
cumvar 0.9 0.01
RUNNING:  0 is-fixated-on-seqc0shape  --  ['shape-macrosaccade-index', 'seqc_0_shape', 'seqc_0_loc', 'task_kind']
len pa bnefore filt this values (var, levs):  task_kind ['prims_on_grid']
pa size, before slicing with variable=task_kind (6, 2381, 1)
... pa size, after slicing with variable=task_kind (6, 2000, 1)
len pa bnefore filt this values (var, levs):  shape-macrosaccade-index [1]
pa size, before slicing with variable=shape-macrosaccade-index (6, 2000, 1)
... pa size, after slicing with variable=shape-macrosaccade-index (6, 462, 1)
FINAL DIMENSION OF DATA (after dimredu, before eucl): (6, 412, 1)
... dataextract_as_distance_matrix_clusters_flex
... computing distance matrices, using distnace: euclidian_unbiased
... rsa_distmat_score_same_diff_by_context
... computing distance matrices, using distnace: euclidian
DIST_NULL_50 2.8631126718008417
fig: /home/kgg/Desktop/eyetracking_analyses/SUBSPACES/Diego-230630/euclidian_dist_plots_macrosacc2/PMv_m/(-0.2, 0

Running PCA
cumvar 0.9 0.01
RUNNING:  0 is-fixated-on-seqc0shape  --  ['shape-macrosaccade-index', 'seqc_0_shape', 'seqc_0_loc', 'task_kind']
len pa bnefore filt this values (var, levs):  task_kind ['prims_on_grid']
pa size, before slicing with variable=task_kind (6, 2381, 1)
... pa size, after slicing with variable=task_kind (6, 2000, 1)
len pa bnefore filt this values (var, levs):  shape-macrosaccade-index [1]
pa size, before slicing with variable=shape-macrosaccade-index (6, 2000, 1)
... pa size, after slicing with variable=shape-macrosaccade-index (6, 462, 1)
FINAL DIMENSION OF DATA (after dimredu, before eucl): (6, 412, 1)
... dataextract_as_distance_matrix_clusters_flex
... computing distance matrices, using distnace: euclidian_unbiased
... rsa_distmat_score_same_diff_by_context
... computing distance matrices, using distnace: euclidian
DIST_NULL_50 2.5550213789669
fig: /home/kgg/Desktop/eyetracking_analyses/SUBSPACES/Diego-230630/euclidian_dist_plots_macrosacc2/PMd_p/(-0.2, 0)/c

...

... rsa_distmat_score_same_diff_by_context
... computing distance matrices, using distnace: euclidian
DIST_NULL_50 2.6700358610107116
fig: /home/kgg/Desktop/eyetracking_analyses/SUBSPACES/Diego-230630/euclidian_dist_plots_macrosacc2/FP_a/(0, 0.2)/color=is-fixated-on-seqc0shape-sub=shape-macrosaccade-index|seqc_0_shape|seqc_0_loc|task_kind-dims=(0, 1)-1.pdf
Saving ...  /home/kgg/Desktop/eyetracking_analyses/SUBSPACES/Diego-230630/euclidian_dist_plots_macrosacc2/FP_a/(0, 0.2)/color=is-fixated-on-seqc0shape-sub=shape-macrosaccade-index|seqc_0_shape|seqc_0_loc|task_kind-dims=(0, 1)-1.pdf
fig: /home/kgg/Desktop/eyetracking_analyses/SUBSPACES/Diego-230630/euclidian_dist_plots_macrosacc2/FP_a/(0, 0.2)/color=is-fixated-on-seqc0shape-sub=shape-macrosaccade-index|seqc_0_shape|seqc_0_loc|task_kind-dims=(2, 3)-1.pdf
Saving ...  /home/kgg/Desktop/eyetracking_analyses/SUBSPACES/Diego-230630/euclidian_dist_plots_macrosacc2/FP_a/(0, 0.2)/color=is-fixated-on-seqc0shape-sub=shape-macrosaccade-index|seqc

... rsa_distmat_score_same_diff_by_context
... computing distance matrices, using distnace: euclidian
DIST_NULL_50 2.916756294125067
fig: /home/kgg/Desktop/eyetracking_analyses/SUBSPACES/Diego-230630/euclidian_dist_plots_macrosacc2/SMA_p/(0, 0.2)/color=is-fixated-on-seqc0shape-sub=shape-macrosaccade-index|seqc_0_shape|seqc_0_loc|task_kind-dims=(0, 1)-1.pdf
Saving ...  /home/kgg/Desktop/eyetracking_analyses/SUBSPACES/Diego-230630/euclidian_dist_plots_macrosacc2/SMA_p/(0, 0.2)/color=is-fixated-on-seqc0shape-sub=shape-macrosaccade-index|seqc_0_shape|seqc_0_loc|task_kind-dims=(0, 1)-1.pdf
fig: /home/kgg/Desktop/eyetracking_analyses/SUBSPACES/Diego-230630/euclidian_dist_plots_macrosacc2/SMA_p/(0, 0.2)/color=is-fixated-on-seqc0shape-sub=shape-macrosaccade-index|seqc_0_shape|seqc_0_loc|task_kind-dims=(2, 3)-1.pdf
Saving ...  /home/kgg/Desktop/eyetracking_analyses/SUBSPACES/Diego-230630/euclidian_dist_plots_macrosacc2/SMA_p/(0, 0.2)/color=is-fixated-on-seqc0shape-sub=shape-macrosaccade-index|s

... rsa_distmat_score_same_diff_by_context
... computing distance matrices, using distnace: euclidian
DIST_NULL_50 3.058289229898219
fig: /home/kgg/Desktop/eyetracking_analyses/SUBSPACES/Diego-230630/euclidian_dist_plots_macrosacc2/SMA_a/(0, 0.2)/color=is-fixated-on-seqc0shape-sub=shape-macrosaccade-index|seqc_0_shape|seqc_0_loc|task_kind-dims=(0, 1)-1.pdf
Saving ...  /home/kgg/Desktop/eyetracking_analyses/SUBSPACES/Diego-230630/euclidian_dist_plots_macrosacc2/SMA_a/(0, 0.2)/color=is-fixated-on-seqc0shape-sub=shape-macrosaccade-index|seqc_0_shape|seqc_0_loc|task_kind-dims=(0, 1)-1.pdf
fig: /home/kgg/Desktop/eyetracking_analyses/SUBSPACES/Diego-230630/euclidian_dist_plots_macrosacc2/SMA_a/(0, 0.2)/color=is-fixated-on-seqc0shape-sub=shape-macrosaccade-index|seqc_0_shape|seqc_0_loc|task_kind-dims=(2, 3)-1.pdf
Saving ...  /home/kgg/Desktop/eyetracking_analyses/SUBSPACES/Diego-230630/euclidian_dist_plots_macrosacc2/SMA_a/(0, 0.2)/color=is-fixated-on-seqc0shape-sub=shape-macrosaccade-index|s

... rsa_distmat_score_same_diff_by_context
... computing distance matrices, using distnace: euclidian
DIST_NULL_50 2.990273386980794
fig: /home/kgg/Desktop/eyetracking_analyses/SUBSPACES/Diego-230630/euclidian_dist_plots_macrosacc2/preSMA_p/(0, 0.2)/color=is-fixated-on-seqc0shape-sub=shape-macrosaccade-index|seqc_0_shape|seqc_0_loc|task_kind-dims=(0, 1)-1.pdf
Saving ...  /home/kgg/Desktop/eyetracking_analyses/SUBSPACES/Diego-230630/euclidian_dist_plots_macrosacc2/preSMA_p/(0, 0.2)/color=is-fixated-on-seqc0shape-sub=shape-macrosaccade-index|seqc_0_shape|seqc_0_loc|task_kind-dims=(0, 1)-1.pdf
fig: /home/kgg/Desktop/eyetracking_analyses/SUBSPACES/Diego-230630/euclidian_dist_plots_macrosacc2/preSMA_p/(0, 0.2)/color=is-fixated-on-seqc0shape-sub=shape-macrosaccade-index|seqc_0_shape|seqc_0_loc|task_kind-dims=(2, 3)-1.pdf
Saving ...  /home/kgg/Desktop/eyetracking_analyses/SUBSPACES/Diego-230630/euclidian_dist_plots_macrosacc2/preSMA_p/(0, 0.2)/color=is-fixated-on-seqc0shape-sub=shape-macrosac

... rsa_distmat_score_same_diff_by_context
... computing distance matrices, using distnace: euclidian
DIST_NULL_50 2.7496314630835745
fig: /home/kgg/Desktop/eyetracking_analyses/SUBSPACES/Diego-230630/euclidian_dist_plots_macrosacc2/preSMA_a/(0, 0.2)/color=is-fixated-on-seqc0shape-sub=shape-macrosaccade-index|seqc_0_shape|seqc_0_loc|task_kind-dims=(0, 1)-1.pdf
Saving ...  /home/kgg/Desktop/eyetracking_analyses/SUBSPACES/Diego-230630/euclidian_dist_plots_macrosacc2/preSMA_a/(0, 0.2)/color=is-fixated-on-seqc0shape-sub=shape-macrosaccade-index|seqc_0_shape|seqc_0_loc|task_kind-dims=(0, 1)-1.pdf
fig: /home/kgg/Desktop/eyetracking_analyses/SUBSPACES/Diego-230630/euclidian_dist_plots_macrosacc2/preSMA_a/(0, 0.2)/color=is-fixated-on-seqc0shape-sub=shape-macrosaccade-index|seqc_0_shape|seqc_0_loc|task_kind-dims=(2, 3)-1.pdf
Saving ...  /home/kgg/Desktop/eyetracking_analyses/SUBSPACES/Diego-230630/euclidian_dist_plots_macrosacc2/preSMA_a/(0, 0.2)/color=is-fixated-on-seqc0shape-sub=shape-macrosa

### Save and plot results

In [31]:
SAVEDIR = f"{SAVEDIR_ANALYSIS}/RESULTS"
import os
import pickle
from pythonlib.tools.pandastools import append_col_with_grp_index
from neuralmonkey.scripts.analy_euclidian_dist_pop_script import plot_all_results
os.makedirs(SAVEDIR, exist_ok=True)

In [32]:
pd.to_pickle(DFRES, f"{SAVEDIR}/DFRES.pkl")

In [33]:
# spit out dfres twinds
twinds = DFRES['twind_analy'].unique()

In [34]:
for t in twinds:
    dir_t = f"{SAVEDIR}/{t}"
    os.makedirs(dir_t, exist_ok=True)
    path = f"{dir_t}/DFRES_{t}.pkl"
    DFRES_t = DFRES[DFRES['twind_analy'] == t].reset_index(drop=True)
    pd.to_pickle(DFRES_t, path)
    plot_all_results(DFRES_t, dir_t)

..Plotting overview bar and scatter plots


/home/kgg/Desktop/pythonlib/pythonlib/tools/snstools.py:25: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(list_text,rotation=rotation, horizontalalignment="right")
/home/kgg/Desktop/pythonlib/pythonlib/tools/snstools.py:25: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(list_text,rotation=rotation, horizontalalignment="right")
/home/kgg/Desktop/pythonlib/pythonlib/tools/snstools.py:25: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(list_text,rotation=rotation, horizontalalignment="right")
/home/kgg/Desktop/pythonlib/pythonlib/tools/snstools.py:25: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(list_text,rotation=rotation, horizontalalignment="right")


..Plotting pairwise for one specific pair of variables:
norm_dist_effect
..Plotting pairwise for one specific pair of variables:
norm_dist_context
..Plotting pairwise for one specific pair of variables:
norm_dist_both
..Plotting pairwise for one specific pair of variables:
generalization_index
..Plotting pairwise for one specific pair of variables:
generalization_index_scaled
..Plotting pairwise for one specific pair of variables:
gen_idx_diff_3
..Plotting pairwise for one specific pair of variables:
gen_idx_diff_4
..Plotting pairwise for one specific pair of variables:
gen_idx_diff_5
..Plotting pairwise for one specific pair of variables:
gen_idx_ratio_1
..Plotting pairwise for one specific pair of variables:
gen_idx_ratio_2
..Plotting pairwise for one specific pair of variables:
gen_idx_ratio_3
..Plotting pairwise for one specific pair of variables:
gen_idx_ratio_4
..Plotting pairwise for one specific pair of variables:
gen_idx_ratio_5
Plotting specific conjucntions heatmaps ... 
...

/home/kgg/Desktop/pythonlib/pythonlib/tools/snstools.py:25: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(list_text,rotation=rotation, horizontalalignment="right")
/home/kgg/Desktop/pythonlib/pythonlib/tools/snstools.py:25: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(list_text,rotation=rotation, horizontalalignment="right")
/home/kgg/Desktop/pythonlib/pythonlib/tools/snstools.py:25: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(list_text,rotation=rotation, horizontalalignment="right")
/home/kgg/Desktop/pythonlib/pythonlib/tools/snstools.py:25: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(list_text,rotation=rotation, horizontalalignment="right")


..Plotting pairwise for one specific pair of variables:
norm_dist_effect
..Plotting pairwise for one specific pair of variables:
norm_dist_context
..Plotting pairwise for one specific pair of variables:
norm_dist_both
..Plotting pairwise for one specific pair of variables:
generalization_index
..Plotting pairwise for one specific pair of variables:
generalization_index_scaled
..Plotting pairwise for one specific pair of variables:
gen_idx_diff_3
..Plotting pairwise for one specific pair of variables:
gen_idx_diff_4
..Plotting pairwise for one specific pair of variables:
gen_idx_diff_5
..Plotting pairwise for one specific pair of variables:
gen_idx_ratio_1
..Plotting pairwise for one specific pair of variables:
gen_idx_ratio_2
..Plotting pairwise for one specific pair of variables:
gen_idx_ratio_3
..Plotting pairwise for one specific pair of variables:
gen_idx_ratio_4
..Plotting pairwise for one specific pair of variables:
gen_idx_ratio_5
Plotting specific conjucntions heatmaps ... 
...

# Tailored plots for specific runs/models

In [45]:
from neuralmonkey.scripts.analy_euclidian_dist_pop_script import compute_all_derived_metrics, _plot_histograms_clean

In [46]:
import pandas as pd
DFRES = pd.read_pickle(f"{SAVEDIR}/DFRES.pkl")

In [47]:
DFRES['animal'] = animal
DFRES['date'] = date

In [48]:
DFRES_new = DFRES[DFRES['twind_analy']==(-0.2, 0)].reset_index(drop=True)

In [49]:
DFRES_ORIG, DFRES_PIVOT_DISTR, DFRES_PIVOT_PAIRWISE, DFRES_PIVOT_YUE, plot_params = compute_all_derived_metrics(DFRES_new)

In [50]:
print(DFRES_ORIG['var_var_others'].unique())

["00|is-fixated-on-seqc0shape|('shape-macrosaccade-index', 'seqc_0_shape', 'seqc_0_loc', 'task_kind')"
 "01|is-fixated-on-seqc0shape|('shape-macrosaccade-index', 'seqc_0_shape', 'seqc_0_loc', 'task_kind')"]


In [51]:
savedir = f"{SAVEDIR}/histograms/pre-fixation/00|is-fixated-on-seqc0shape-vlPFC"
import os
os.makedirs(savedir, exist_ok=True)

In [52]:
_plot_histograms_clean(DFRES_ORIG, ["00|is-fixated-on-seqc0shape|('shape-macrosaccade-index', 'seqc_0_shape', 'seqc_0_loc', 'task_kind')"],
                      savedir, effect_context='diff|same', dat_level='distr', bregions_plot=['vlPFC_a', 'vlPFC_p', 'M1_m'])

/home/kgg/Desktop/pythonlib/pythonlib/tools/snstools.py:25: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(list_text,rotation=rotation, horizontalalignment="right")
/home/kgg/Desktop/pythonlib/pythonlib/tools/snstools.py:25: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(list_text,rotation=rotation, horizontalalignment="right")
/home/kgg/Desktop/pythonlib/pythonlib/tools/snstools.py:25: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(list_text,rotation=rotation, horizontalalignment="right")
/home/kgg/Desktop/pythonlib/pythonlib/tools/snstools.py:25: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(list_text,rotation=rotation, horizontalalignment="right")


Saving to:  /home/kgg/Desktop/eyetracking_analyses/SUBSPACES/Diego-230630/euclidian_dist_plots_macrosacc2/RESULTS/histograms/pre-fixation/00|is-fixated-on-seqc0shape-vlPFC/brain_schematics/brainschem-var_var_others-dist_norm_95-norm_None-.pdf
Saving to:  /home/kgg/Desktop/eyetracking_analyses/SUBSPACES/Diego-230630/euclidian_dist_plots_macrosacc2/RESULTS/histograms/pre-fixation/00|is-fixated-on-seqc0shape-vlPFC/brain_schematics_each_var/brainschem-None-dist_norm_95-norm_None-00|is-fixated-on-seqc0shape|('shape-macrosaccade-index', 'seqc_0_shape', 'seqc_0_loc', 'task_kind').pdf


In [53]:
savedir = f"{SAVEDIR}/histograms/pre-fixation/01|is-fixated-on-seqc0shape-vlPFC"
os.makedirs(savedir, exist_ok=True)

In [54]:
_plot_histograms_clean(DFRES_ORIG, ["01|is-fixated-on-seqc0shape|('shape-macrosaccade-index', 'seqc_0_shape', 'seqc_0_loc', 'task_kind')"],
                      savedir, effect_context='diff|same', dat_level='distr', bregions_plot=['vlPFC_a', 'vlPFC_p', 'M1_m'])

/home/kgg/Desktop/pythonlib/pythonlib/tools/snstools.py:25: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(list_text,rotation=rotation, horizontalalignment="right")
/home/kgg/Desktop/pythonlib/pythonlib/tools/snstools.py:25: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(list_text,rotation=rotation, horizontalalignment="right")
/home/kgg/Desktop/pythonlib/pythonlib/tools/snstools.py:25: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(list_text,rotation=rotation, horizontalalignment="right")
/home/kgg/Desktop/pythonlib/pythonlib/tools/snstools.py:25: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(list_text,rotation=rotation, horizontalalignment="right")


Saving to:  /home/kgg/Desktop/eyetracking_analyses/SUBSPACES/Diego-230630/euclidian_dist_plots_macrosacc2/RESULTS/histograms/pre-fixation/01|is-fixated-on-seqc0shape-vlPFC/brain_schematics/brainschem-var_var_others-dist_norm_95-norm_None-.pdf
Saving to:  /home/kgg/Desktop/eyetracking_analyses/SUBSPACES/Diego-230630/euclidian_dist_plots_macrosacc2/RESULTS/histograms/pre-fixation/01|is-fixated-on-seqc0shape-vlPFC/brain_schematics_each_var/brainschem-None-dist_norm_95-norm_None-01|is-fixated-on-seqc0shape|('shape-macrosaccade-index', 'seqc_0_shape', 'seqc_0_loc', 'task_kind').pdf


In [36]:
savedir = f"{SAVEDIR}/histograms/pre-fixation/05|is-fixated-on-seqc0shape"
os.makedirs(savedir, exist_ok=True)

In [37]:
_plot_histograms_clean(DFRES_ORIG, ["05|is-fixated-on-seqc0shape|('shape-macrosaccade-index', 'seqc_0_shape', 'seqc_0_loc', 'task_kind')"],
                      savedir, effect_context='diff|same', dat_level='distr', bregions_plot=['vlPFC_a', 'vlPFC_p', 'M1_m'])

/home/kgg/Desktop/pythonlib/pythonlib/tools/snstools.py:25: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(list_text,rotation=rotation, horizontalalignment="right")
/home/kgg/Desktop/pythonlib/pythonlib/tools/snstools.py:25: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(list_text,rotation=rotation, horizontalalignment="right")
/home/kgg/Desktop/pythonlib/pythonlib/tools/snstools.py:25: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(list_text,rotation=rotation, horizontalalignment="right")
/home/kgg/Desktop/pythonlib/pythonlib/tools/snstools.py:25: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(list_text,rotation=rotation, horizontalalignment="right")


Saving to:  /home/kgg/Desktop/eyetracking_analyses/SUBSPACES/Diego-230630/euclidian_dist_plots/RESULTS/histograms/pre-fixation/05|is-fixated-on-seqc0shape/brain_schematics/brainschem-var_var_others-dist_norm_95-norm_None-.pdf
Saving to:  /home/kgg/Desktop/eyetracking_analyses/SUBSPACES/Diego-230630/euclidian_dist_plots/RESULTS/histograms/pre-fixation/05|is-fixated-on-seqc0shape/brain_schematics_each_var/brainschem-None-dist_norm_95-norm_None-05|is-fixated-on-seqc0shape|('shape-macrosaccade-index', 'seqc_0_shape', 'seqc_0_loc', 'task_kind').pdf


# Pre/post euclidian coefficient comparisons

In [ ]:
import pandas as pd
import numpy as np
from neuralmonkey.scripts.analy_euclidian_dist_pop_script import compute_all_derived_metrics
import matplotlib.pyplot as plt

In [ ]:
#path = f"/home/lucas/Dropbox/SCIENCE/FREIWALD_LAB/DATA/DFallpa_Diego_230929_RULE_BASE_stroke.pkl"
animal = "Diego"
date = "230629"

In [ ]:
list_list_coefs = {}

In [ ]:
models = []

In [ ]:
for twind in [(-0.2, 0), (0, 0.2)]:
    path = f"/tmp/{animal}_{date}/euclidian_240506/RESULTS/{twind}/DFRES_{twind}.pkl"
    DFRES = pd.read_pickle(path)
    DFRES['twind_analy'] = DFRES['twind']
    DFRES['var_var_others'] = DFRES['var'].astype(str) + DFRES['var_others'].astype(str)

    DFRES_ORIG, DFRES_PIVOT_DISTR, DFRES_PIVOT_PAIRWISE, DFRES_PIVOT_YUE, plot_params = compute_all_derived_metrics(DFRES)

    models = DFRES_ORIG['var_var_others'].unique()
    list_coefs = {}
    for m in models:
        coefs = {}
        print(m)
        DFRES_t = DFRES_ORIG[DFRES_ORIG['var_var_others']==m]
        bregions = DFRES_t['bregion'].unique()
        for b in bregions:
            DFRES_tt = DFRES_t[(DFRES_t['bregion']==b) & (DFRES_t['dat_level']=='pts_yue_diff') & (DFRES_t['shuffled']==False) & (DFRES_t['effect_context']=='diff|same')]
            coefs[b] = np.mean(DFRES_tt['dist_norm_95'])
        list_coefs[m] = coefs
    list_list_coefs[str(twind)] = list_coefs

In [ ]:
DFRES_tt

In [ ]:
DFRES_tt.columns

In [ ]:
list_list_coefs

In [ ]:
models

In [ ]:
twind_pre = "(-0.2, 0)"
twind_post = "(0, 0.2)"

In [ ]:
fig, axes = plt.subplots(len(models), 1, squeeze=False, figsize=(10, 5*len(models)))
for i, m in enumerate(models):
    ax = axes[i,0]
    ax.set_xlim([-0.05, 0.15])
    ax.set_ylim([-0.05, 0.15])
    ax.set_box_aspect(1)

    x = list(list_list_coefs[twind_pre][m].values())
    y = list(list_list_coefs[twind_post][m].values())
    n = list(list_list_coefs[twind_pre][m].keys())

    ax.scatter(x, y)
    ax.plot([-0.1, 0.3], [-0.1, 0.3])
    ax.axhline(0)
    ax.axvline(0)
    ax.set_xlabel("pre-fixation (-0.2, 0)")
    ax.set_ylabel("post-fixation (0, 0.2)")
    ax.set_title(m)

    for i, txt in enumerate(n):
        ax.annotate(txt, (x[i], y[i]))